# Project Notebook: Optimizing DataFrames and Processing in Chunks

## 1. Introduction 

In this project, we'll practice working with chunked dataframes and optimizing a dataframe's memory usage. We'll be working with financial lending data from Lending Club, a marketplace for personal loans that matches borrowers with investors. You can read more about the marketplace on its website.

The Lending Club's website lists approved loans. Qualified investors can view the borrower's credit score, the purpose of the loan, and other details in the loan applications. Once a lender is ready to back a loan, it selects the amount of money it wants to fund. When the loan amount the borrower requested is fully funded, the borrower receives the money, minus the origination fee that Lending Club charges.

We'll be working with a dataset of loans approved from 2007-2011 (https://bit.ly/3H2XVgC). We've already removed the desc column for you to make our system run more quickly.

If we read in the entire data set, it will consume about 67 megabytes of memory. Let's imagine that we only have 10 megabytes of memory available throughout this project, so you can practice the concepts you learned in the last two lessons.

**Tasks**

1. Read in the first five lines from `loans_2007.csv` (https://bit.ly/3H2XVgC) and look for any data quality issues.

2. Read in the first 1000 rows from the data set, and calculate the total memory usage for these rows. Increase or decrease the number of rows to converge on a memory usage under five megabytes (to stay on the conservative side).

In [2]:
from pandas.core.arrays.categorical import no_default
# Importing pandas
import pandas as pd
pd.options.display.max_columns = 99

# Your code goes here
loans_head = pd.read_csv('https://bit.ly/3H2XVgC', nrows=5,)
loans_sample = pd.read_csv('https://bit.ly/3H2XVgC', nrows=1000,)
#print(loans_sample.info(memory_usage='deep'))
print(f'memory_usage for 1000 rows: {loans_sample.memory_usage(deep=True).sum()/(1024*1024)}')

memory_usage for 1000 rows: 1.5273666381835938


In [3]:
#1000 rows have memory usage of 1.5MB
#increament the number of rows in 50 to have memory usage of close to 5MB
chunk_memory = 0
no_rows =2500
while(chunk_memory <5):
  no_rows += 50
  loans_sample = pd.read_csv('https://bit.ly/3H2XVgC', nrows=no_rows,)
  chunk_memory = loans_sample.memory_usage(deep=True).sum()/(1024*1024)

print(f'no of rows for just 5MB memory usage: {no_rows-50}')
print(f'chunk memory for the max rows {chunk_memory}')
no_rows = no_rows-50

no of rows for just 5MB memory usage: 3250
chunk memory for the max rows 5.038409233093262


##chunk size
we will be working with chunk size of 3250 rows

## 2. Exploring the Data in Chunks

Let's familiarize ourselves with the columns to see which ones we can optimize. In the first lesson, we explored column types by reading in the full dataframe. In this project, let's try to understand the column types better while using dataframe chunks.

**Tasks**

For each chunk:
* How many columns have a numeric type? 
* How many have a string type?
* How many unique values are there in each string column? How many of the string columns contain values that are less than 50% unique?
* Which float columns have no missing values and could be candidates for conversion to the integer type?
* Calculate the total memory usage across all of the chunks.

##observation
### Each float column has atleast 3 na values for the whole dataset. use a filter columns <100 na values, drop na and convert the float column to integer

In [5]:
from pandas.core.tools import numeric
# Your code goes here
chunk_iter = pd.read_csv('https://bit.ly/3H2XVgC', chunksize=no_rows,)

numeric_cols = []
float_cols_na = []
string_cols = []
string_cols_uniqueness = []
total_string_cols_uniqueness = pd.Series()
chunk_memory_usage = []
for chunk in chunk_iter:
  #the number of numeric columns for each chunk
  #the 'id' column number, numeric string and non-numeric string
  chunk['id'] = pd.to_numeric(chunk['id'], errors='coerce') #non-numeric string converted to null values
  chunk = chunk.dropna(axis=0, subset=['id']) #drop row where id is missing
  chunk_numeric_cols = chunk.select_dtypes(exclude=['object']).shape[1]
  numeric_cols.append(chunk_numeric_cols)
  
  #check for any missing value for each chunk numeric columns
  chunk_float_cols = chunk.select_dtypes(include=['float64'])
  float_cols_na.append(chunk_float_cols.isnull().sum())

  #the number of string columns for each chunk
  chunk_string_cols = chunk.select_dtypes(include=['object']).shape[1]
  string_cols.append(chunk_string_cols)

  #unique values in each string column and check if they are <50% of number of values in the column
  chunk_string_cols_unique = chunk.select_dtypes(include=['object']).nunique() #pd.series with unique values per col
  chunk_string_cols_count = chunk.select_dtypes(include=['object']).count() #total no of items in col
  string_cols_uniqueness.append((100*chunk_string_cols_unique/chunk_string_cols_count)) #add each chunk series to a list

  #memory usage for the chunk
  chunk_memory_usage.append(chunk.memory_usage(deep=True))

#
#concatenate the series in the list to a series
total_string_cols_uniqueness = pd.concat(string_cols_uniqueness)
total_string_cols_uniqueness = total_string_cols_uniqueness.groupby(total_string_cols_uniqueness.index).mean()
categorise_string_cols = list((total_string_cols_uniqueness[total_string_cols_uniqueness <50]).index)

#combine all numeric columns missing values
total_float_cols_na = pd.concat(float_cols_na)
total_float_cols_na = total_float_cols_na.groupby(total_float_cols_na.index).sum()
zero_na_float_col = list((total_float_cols_na[total_float_cols_na == 0 ]).index)
float_col_with_na = list((total_float_cols_na[total_float_cols_na > 0 ]).index)

#sum up the memory usage
total_memory_usage = pd.concat(chunk_memory_usage).sum()/(1024*1024)

print(f'\nzero na float columns:\n {zero_na_float_col}')
print(f'\nfloat columns with null value:\n {float_col_with_na}')
print(f'\nmissing values in dataset:\n {total_float_cols_na}')
print(f'\nchunks:{len(numeric_cols)} chunks: numeric columns for the chunks: {numeric_cols}')
print(f'\nchunks:{len(string_cols)} chunks: string columns for the chunks: {string_cols}')
print(f'\npercentage of unique values in the columns: \n {total_string_cols_uniqueness}')
print(f'\ncolumns that can be converted to category dtype: \n cols no:{len(categorise_string_cols)} : {categorise_string_cols}')
print(f'\nMemory usage: {total_memory_usage}MB')

  


<ipython-input-5-e3abb922cbcc>:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  total_string_cols_uniqueness = pd.Series()



zero na float columns:
 ['collection_recovery_fee', 'dti', 'funded_amnt', 'funded_amnt_inv', 'id', 'installment', 'last_pymnt_amnt', 'loan_amnt', 'member_id', 'out_prncp', 'out_prncp_inv', 'policy_code', 'recoveries', 'revol_bal', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp']

float columns with null value:
 ['acc_now_delinq', 'annual_inc', 'chargeoff_within_12_mths', 'collections_12_mths_ex_med', 'delinq_2yrs', 'delinq_amnt', 'inq_last_6mths', 'open_acc', 'pub_rec', 'pub_rec_bankruptcies', 'tax_liens', 'total_acc']

missing values in dataset:
 acc_now_delinq                  29
annual_inc                       4
chargeoff_within_12_mths       145
collection_recovery_fee          0
collections_12_mths_ex_med     145
delinq_2yrs                     29
delinq_amnt                     29
dti                              0
funded_amnt                      0
funded_amnt_inv                  0
id                               0
inq_last_6mths   

## 3. Optimizing String Columns

We can achieve the greatest memory improvements by converting the string columns to a numeric type. Let's convert all of the columns where the values are less than 50% unique to the category type, and the columns that contain numeric values to the `float` type.

While working with dataframe chunks:
* Determine which string columns you can convert to a numeric type if you clean them. For example, the `int_rate` column is only a string because of the % sign at the end.
* Determine which columns have a few unique values and convert them to the category type. For example, you may want to convert the grade and `sub_grade` columns.
Based on your conclusions, perform the necessary type changes across all chunks. * Calculate the total memory footprint, and compare it with the previous one.

In [6]:
# Your code goes here
chunk_iter = pd.read_csv('https://bit.ly/3H2XVgC', chunksize=no_rows,)
#columns to be categorised, remove columns to be converted float dtype
categorise_string_cols.remove('int_rate')
categorise_string_cols.remove('revol_util')

string_optimised_mem_usage = []
for chunk in chunk_iter:
  #cols 'int_rate' and 'revol_util' can be converted to float by removing %
  chunk['int_rate'] = chunk['int_rate'].str.replace('%', '').astype('float')
  chunk['revol_util'] = chunk['revol_util'].str.replace('%', '').astype('float')
  chunk['int_rate'] = pd.to_numeric(chunk['int_rate'], downcast='float')
  chunk['revol_util'] = pd.to_numeric(chunk['revol_util'], downcast='float')
  #categorise columns with unique values <50% of total items in the column
  chunk[categorise_string_cols] = chunk[categorise_string_cols].astype('category')

  #memory usage of the chunk optimised strings columns
  string_optimised_mem_usage.append(chunk.memory_usage(deep=True))
  

#sum up the memory usage
total_string_optimised_mem = pd.concat(string_optimised_mem_usage).sum()/(1024*1024)
print(f'\nmem usage with string columns optimisation\n {total_string_optimised_mem}MB')
print(f'memory usage improved {total_memory_usage/total_string_optimised_mem}times')



mem usage with string columns optimisation
 18.75536346435547MB
memory usage improved 3.485781044891537times


##Observation:
### memory utilisation after string columns optimisation reduced from 65MB to 18MB

## 4. Optimizing Numeric Columns

It looks like we were able to realize some powerful memory savings by converting to the category type and converting string columns to numeric ones.

Now let's optimize the numeric columns using the `pandas.to_numeric()` function.

**Tasks**

While working with dataframe chunks:
* Identify float columns that contain missing values, and that we can convert to a more space efficient subtype.
* Identify float columns that don't contain any missing values, and that we can convert to the integer type because they represent whole numbers.
* Based on your conclusions, perform the necessary type changes across all chunks.
* Calculate the total memory footprint and compare it with the previous one.




In [7]:
#Your code goes here

chunk_iter = pd.read_csv('https://bit.ly/3H2XVgC', chunksize=no_rows,)

numeric_optimised_mem_usage = []
total_int_na = [] 

#iterate through df columns and downcast dtype
def mem_optimisation(df, columns, convert_dtype):
  for c in columns:
    df[c] = pd.to_numeric(df[c], downcast=convert_dtype, errors='coerce')
  return df

for chunk in chunk_iter:
  #convert the id column object to float and remove null value
  chunk['id'] = pd.to_numeric(chunk['id'], errors='coerce') #non-numeric string converted to null values
  chunk = chunk.dropna(axis=0, subset=['id']) #drop row where id is missing
  
  #convert the float columns with zero null value to int
  chunk = mem_optimisation(chunk, zero_na_float_col, 'integer')
  chunk = mem_optimisation(chunk, float_col_with_na, 'float')

  #memory usage of the chunk optimised strings columns
  numeric_optimised_mem_usage.append(chunk.memory_usage(deep=True))

#sum up the memory usage
total_numeric_optimised_mem = pd.concat(numeric_optimised_mem_usage).sum()/(1024*1024)
print(f'\nmem usage with numeric columns optimisation\n {total_numeric_optimised_mem}MB')
print(f'memory usage improved {total_memory_usage/total_numeric_optimised_mem}times')



mem usage with numeric columns optimisation
 62.038339614868164MB
memory usage improved 1.0538175402494692times


<ipython-input-7-09892fddb9bd>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[c] = pd.to_numeric(df[c], downcast=convert_dtype, errors='coerce')


## Next Steps

We've practiced optimizing a dataframe's memory footprint and working with dataframe chunks. Here's an idea for some next steps:

Create a function that automates as much of the work you just did as possible, so that you could use it on other Lending Club data sets. This function should:

* Determine the optimal chunk size based on the memory constraints you provide.

* Determine which string columns can be converted to numeric ones by removing the `%` character.

* Determine which numeric columns can be converted to more space efficient representations.


In [8]:
# Your code goes here
#Determine the optimal chunk size based on the memory constraints you provide.  
def chunk_size(csv_file, available_mem, start_chunk_size, chunk_step):
  no_rows = start_chunk_size
  chunk_memory =0
  while(chunk_memory < available_mem):
    no_rows += chunk_step
    chunk = pd.read_csv(csv_file, nrows=no_rows,)
    chunk_memory = chunk.memory_usage(deep=True).sum()/(1024*1024)
    
  #reduce by 1
  #no_rows -=chunk_step
  #get the chunks
  #chunk_iter = pd.read_csv(csv_file, chunk_size=no_rows,)#

  return no_rows-chunk_step

no_rows = chunk_size('https://bit.ly/3H2XVgC', 5, 3000, 50)
print(no_rows)



3250


In [9]:
from IPython.utils.py3compat import iteritems
#Determine which string columns can be converted to numeric ones by removing the % character.
def string_to_numeric(df, ch):
  columns_with_ch = []
  for r,c in df.iteritems():
    # ch_found = c.str.contains(ch)
    # print(ch_found)
    #check if column has % character, remove it and try to convert to numeric, 
    #if it had only % character, no error raised but if other string character error will be raised
    if  c.str.contains(ch).any():
      try:
        c = c.str.replace('%', '')
        c = pd.to_numeric(c, errors='raise')
        columns_with_ch.append(r)
      except(ValueError):
        #dont add columns to the list
        continue
  print(columns_with_ch)

loans = pd.read_csv('https://bit.ly/3H2XVgC')
string_cols = loans.select_dtypes(include='object')
string_to_numeric(string_cols, '%')


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


['int_rate', 'revol_util']


In [10]:
#Determine which numeric columns can be converted to more space efficient representations
#numeric cols can be int64(no missing value) or float64(allow missing value)
def numeric_cols_optimise(df): 
  #select float type columns and check the number of missing value per columns
  numeric_cols = df.select_dtypes(exclude=['object'])
  cols_missing_value = numeric_cols.isnull().sum()
  #float columns with no null values can be converted to int type
  int_cols = list(cols_missing_value[cols_missing_value == 0].index)
  float_cols = list(cols_missing_value[cols_missing_value >0].index)

  return int_cols, float_cols

#
loans = pd.read_csv('https://bit.ly/3H2XVgC')
loans['id']=pd.to_numeric(loans['id'], errors='coerce')
loans.dropna(axis=0, subset=['id'], inplace=True)

x,y = numeric_cols_optimise(loans)
print(f'int columns: {x}')
print(f'float columns: {y}')




int columns: ['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'installment', 'dti', 'revol_bal', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'policy_code']
float columns: ['annual_inc', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'total_acc', 'collections_12_mths_ex_med', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'pub_rec_bankruptcies', 'tax_liens']
